In [2]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator

import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [3]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

# # load the autoreload extension
# %load_ext autoreload
# # Set extension to reload modules every time before executing code
# %autoreload 2

In [4]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

In [4]:
## !jupyter nbconvert budget_strategies.ipynb --no-input --no-prompt --to pdf
# os.system("jupyter nbconvert budget_strategies.ipynb --no-input --no-prompt --to pdf")
# os.system("jupyter nbconvert budget_strategies.ipynb --config ~/.jupyter/jupyter_nbconvert_config.py --to slides")

# Prepare models/datasets

#### Load the pretrained models "lstm" (entire_model_1m_20interactions.pt) and "pooling" (pooling_model_1m_20interactions.pt)

In [5]:
# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

#### Get the dataset Movielens with the variant 1M. Then divide it into a training set and a testing set. And finally this code limits the length of each sequence of elements in the 2 sets to 20.

In [6]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset 
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

#### 1. Compute the similarity matrix based on cosine similarity
#### 2. Compute the similarity matrix based on Jaccard similarity

In [7]:
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

  0%|          | 0/6040 [00:00<?, ?it/s]

# Various implemented Strategies

In [8]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget, negative_mode,  model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10
        self.negative_mode = negative_mode
        self.budget = init_budget

    #Must be implemented by subclasses. Used to select the next item to recommand to the user.
    def next_comb(self, reverse=False):
        raise NotImplementedError

    
    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    #Method to reset the costs of the last recommended combination
    def reset_costs(self):
        self.last_comb_cost = 0

    #Returns the initial budget
    def get_init_budget(self):
        return self.budget

### RandomSelection
#### This class is a subclass of the "BaseStrategy" class, representing a random item selection strategy for the sequential recommendation task.

In [9]:
class RandomSelection(BaseStrategy):
    class_name = 'Random'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, negative_mode):
        super().__init__(item, interactions, max_sequence_length, init_budget, negative_mode)

    # The _next_item method selects a random integer between 1 and the maximum length of the sequence. 
    # It checks if the integer is already in the set of visited integers, and if it is, selects another integer until a non-visited integer is found. 
    # Finally, the selected integer is added to the set of visited integers, and returned.
    def _next_item(self):
        self.budget -= 1
        if self.negative_mode:
            return self._next_item_negative_mode()
        else:
            return self._next_item_positive_mode()
    
    def _next_item_negative_mode(self):
        length = self.dataset.num_items
        number = random.sample(range(1, pow(2, length)), 1)[0]
        while number in self.visited_ or self.original_interactions:
            number = random.sample(range(1, pow(2, length)), 1)[0]
        self.visited_.add(number)
        return number

    def _next_item_positive_mode(self):
        length = self.max_length
        number = random.sample(range(1, pow(2, length)), 1)[0]
        while number in self.visited_:
            number = random.sample(range(1, pow(2, length)), 1)[0]
        self.visited_.add(number)
        return number
    #The next_comb method generates a new sequence by removing items at positions indicated by the binary digits of the integer returned 
    # by _next_item from the original sequence of interactions. 
    # The resulting sequence, along with the current budget, is returned as a tuple.
    def next_comb(self, reverse=False):
        number = self._next_item()

        bits = self._get_pos(number)
        if self.negative_mode : 
            seq = np.add(self.original_interactions, bits)
        else : 
            seq = np.delete(self.original_interactions, bits)

        return (seq, self.budget)

### MostSimilarSelection
#### This class is a subclass of the "BaseStrategy" class, it is used to select the most similar item to the current item based on the similarity measure defined by the sim_type parameter.

In [10]:
class MostSimilarSelection(BaseStrategy):
    class_name = 'Sim-Matrix'

    supported_sim_matrix = {
        'pooling': pooling_sims_matrix,
        'jaccard': jaccard_sims_matrix
    }

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, negative_mode, sim_type='pooling'):
        super().__init__(item, interactions, max_sequence_length, init_budget, negative_mode)

        self.visited_.add(0)
        self.reverse_checks = []
        self.is_materialized = False

        self._get_sim_ranking(sim_type)

    # The next_comb method returns the next combination of items to be recommended. 
    # If reverse is True, it returns a combination that has been previously seen, otherwise it returns a new combination. The method also updates the visited set.
    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
            selected_item_indices = self.reverse_checks.pop(
                random.randrange(len(self.reverse_checks)) if self.random_pick else 0
            ) if len(self.reverse_checks) else []
        else:
            self.visited_.add(max(self.visited_) + 1)
            selected_item_indices = np.where(np.isin(
                self.rk_items,
                list(set(self.rk_items).difference(set(self.visited_)))
            ))[0]
        seq = self.original_interactions[selected_item_indices] if len(selected_item_indices) else None
        return seq
    
    # The _get_sim_ranking method calculates the similarity ranking between the target item and the other items in the interaction history 
    # based on the similarity measure defined by the sim_type parameter.
    def _get_sim_ranking(self, sim_type):
        if self.negative_mode:
            ranked_items = st.rankdata(-self.supported_sim_matrix[sim_type][self.target_item, self.original_interactions])
        else:
            ranked_items = st.rankdata(self.supported_sim_matrix[sim_type][self.target_item, self.original_interactions])
        self.rk_items = self.max_length - ranked_items + 1

    # The _materialize_list method generates a list of all possible combinations of items that have not been visited, 
    # except for the empty combination and the combination that contains all items. 
    # It stores these combinations in the reverse_checks list to be used later in the next_comb method when reverse is True.
    def _materialize_list(self):
        if not self.is_materialized:
            psize = len(self.visited_) - 1  # do not consider initial added zero value
            # do not take account none/all excluded interacted items
            prods = sorted(list(map(list, itertools.product([0, 1], repeat=psize)))[1:-1], key=sum)
#             last_item_indices = np.where(np.isin(
#                 self.rk_items,
#                 list(set(self.rk_items).difference(set(self.visited_)))
#             ))

            lvisited_ = np.asarray(list(self.visited_))[1:]
            for p in prods:
                self.reverse_checks.append(np.where(np.isin(
                    self.rk_items,
                    list(set(self.rk_items).difference(lvisited_[np.nonzero(np.multiply(p, lvisited_))])))
                ))

            self.is_materialized = True

### MostSimilarSelectionByJaccard
#### This class inherits from the MostSimilarSelection class. This class specifies the jaccard matrix method as similarity measure. 

In [11]:
class MostSimilarSelectionByJaccard(MostSimilarSelection):
    class_name = 'Jaccard-on-Sim-Matrix'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model, 'jaccard')

### RandomMostSimilarSelection
#### This class inherits from the MostSimilarSelection class. 

In [12]:
class RandomMostSimilarSelection(MostSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.random_pick = True 
        #The random_pick attribute is used in the parent class's next_comb method to determine whether to pick a random item from the list 
        # of reverse checks or pick the first one. When random_pick is True, a random item is picked.

### LossSimilarSelection
#### This class inherits frome the BaseStrategy class. This class defines a search strategy for selecting items based on their similarity to previously selected items, while also considering their loss (difference between predicted and actual values) in a ranking problem.

In [13]:
class LossSimilarSelection(BaseStrategy):
    class_name = 'BFS'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, early_term=False):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), StaticVars.INT_MAX, 0))

        self.thres = len(self.original_interactions) + 1
        self.early_termination = early_term

    # Helper : This method is called whenever a solution is found for the current 
    # mask of the items. It computes the loss of the solution and updates the queue 
    # accordingly.
    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    # Helper : 
    def _next_item(self):
        mask, t_score, is_solved = self.q.dequeue()
        while self.early_termination and sum(mask) == self.thres:
            q_data = self.q.dequeue()
            if q_data is None: break

            mask, t_score, is_solved = q_data

        if is_solved == 2:
            t_score, kth_score = self.get_score(mask)

            if (t_score / kth_score) < 1: self.thres = sum(mask)

        return (is_solved, mask, self.budget)

    # The next_comb method returns the next combination of items and the remaining budget.
    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q.size() > 0:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    # This method computes the loss of the solution. If the solution is not yet solved, it searches for the next combination of items 
    # to evaluate by calling the search method. 
    # If the solution is solved, it searches for the previous combination of items by calling the search method with forward=False.
    def compute_loss(self, is_solved=False):
        self.last_comb_cost = 0

        if not is_solved: self.search(forward=True, s=is_solved)
        else: self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    # The get_score method returns the score of an item based on its predicted value and its rank among the top-k items.
    def get_score(self, d):
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

        self.budget -= 1
        # predict next top-k items about to be selected
        preds = self.model.predict(perm)
        if self.negative_mode:
            preds[perm] = StaticVars.FLOAT_MAX
        else:
            preds[perm] = -StaticVars.FLOAT_MAX
        rk_data = st.rankdata(-preds, method='ordinal')

        return (preds[self.target_item], preds[(rk_data == self.top_k).nonzero()][0])

    # The add method adds a new combination to the queue and updates the visited set.
    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

            if not s:
                t_score, kth_score = self.get_score(d)

                if self.q.size() == 0: self.q.enqueue((d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))

                if t_score < self.q.get(0)[1]:  # get only the assigned score
                    self.q.setter(0, (d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))
            else:
                self.q.enqueue((d.copy(), StaticVars.INT_MAX, 2))

            self.visited_.add(mask_to_int)

### DFSwithLossSelection
#### This class inherits from LossSimilarSelection class. It performs a depth-first search with loss selection to recommend items to users.

In [14]:
class DFSwithLossSelection(LossSimilarSelection):
    class_name = 'DFS'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.candidate_solutions = Stack()

    # Helper : it calculates the losses of each item in the interactions and sorts them in ascending order. 
    # It then sets the mask of the interactions array so that the item with the lowest loss is masked.
    def _get_sim_ranking(self):
        res = self.compute_losses()

        m_mask = np.ma.getmask(self.ma_arr).copy()
        if self.negative_mode:
            m_mask[max(res, key=lambda item: item[0])[1]] = True
        else:
            m_mask[min(res, key=lambda item: item[0])[1]] = True
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if np.sum(m_mask) > 1:
            res = self.compute_losses(inv_mask=True)

            for idx in sorted(res, key=lambda item: item[0], reverse=True):
                m_mask[idx[1]] = False
                self.candidate_solutions.push(m_mask.copy())
                m_mask[idx[1]] = True

        m_mask = self.candidate_solutions.pop()
        while m_mask is not None and int(''.join(map(str, m_mask.astype(int))), 2) in self.visited_:
            m_mask = self.candidate_solutions.pop()

        self.visited_.add(0 if m_mask is None else int(''.join(map(str, m_mask.astype(int))), 2))
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True if m_mask is None else m_mask)

### RandomLossSimilarSelection
#### This class inherits from LossSimilarSelection class.

In [15]:
class RandomLossSimilarSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.random_pick = True

### FixedRankingLossSimilarSelection
##### This class is a subclass of LossSimilarSelection. It implements a strategy for selecting the next item based on the fixed ranking of the similarity between items. The strategy implemented by this class is to recommend items in a fixed order based on their similarity score. The order is determined by the ranking of the similarity matrix. The method next_comb is used to retrieve the next combination of items in the fixed order. It selects the next item based on a precomputed fixed ranking of the items in the candidate list, which is obtained by sorting the items according to their similarity loss. The next item to be removed is the one with the highest rank. This means that the sequence of items to be removed is deterministic and fixed in advance.

In [16]:
class FixedRankingLossSimilarSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.rk_items = []
        self._get_sim_ranking()

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
        else:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            m_mask[self.rk_items.pop(0)] = True
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        seq = np.ma.compressed(self.ma_arr)
        return seq if len(seq) else None

    def _get_sim_ranking(self):
        res = self.compute_losses()

        ranked_items = np.asarray(res).argsort(axis=0)
        self.rk_items = [item[0] for item in ranked_items]

### BestFSLossSelection 
#### This class inherits from LossSimilarSelection. It implements a strategy for selecting the best candidate solutions by using a best-first search algorithm. By using a best-first search strategy, this class is able to explore promising candidate solutions first and therefore is expected to find better solutions faster than other strategies like random selection.

In [17]:
import heapq as hq


class BestFSLossSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.set = set()
        self.tiebraker = itertools.count()

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

### TopDownBestFSLossSelection
#### This class inherits from LossSimilarSelection class. The strategy implemented here is to select the most promising combinations of items in a top-down fashion based on their scores. The goal is probably to reduce the search space and speed up the selection process.

In [18]:
import heapq as hq


class TopDownBestFSLossSelection(LossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.best_score_per_cardinality = [StaticVars.FLOAT_MAX] * self.max_length
        self.set = set()
        self.tiebraker = itertools.count()

    def set_score(self, cardinality, target_score, kth_score):
        reverse_search = False
        score = target_score / kth_score
        if score < self.best_score_per_cardinality[cardinality]:
            self.best_score_per_cardinality[cardinality] = score

            if score > 1.0: reverse_search = True

        return reverse_search

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

### DFSwithFixedRankingLossSelection
#### It inherits from FixedRankingLossSimilarSelection. It provides a strategy for selecting items based on the fixed ranking loss similarity metric using a depth-first search algorithm. It generates the next combination of items to be considered for selection by either removing the first item from the ranking list or by materializing the list of all possible combinations of the remaining items.

In [19]:
class DFSwithFixedRankingLossSelection(FixedRankingLossSimilarSelection):

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
        else:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            m_mask[self.rk_items.pop(0)] = True
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        seq = np.ma.compressed(self.ma_arr)
        return seq if len(seq) else None

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if np.sum(m_mask) > 1:
            res = self.compute_losses(inv_mask=True)

            if not self.is_materialized:
                for idx in sorted(res, key=lambda item: item[0]):
                    m_mask[idx[1]] = False
                    self.reverse_checks.append(m_mask.copy())
                    m_mask[idx[1]] = True

                self.is_materialized = True
            else:
                m_mask[min(res, key=lambda item: item[0])[1]] = False
                self.reverse_checks.insert(0, m_mask)

        m_mask = self.reverse_checks.pop(
            random.randrange(len(self.reverse_checks)) if self.random_pick else 0
        ) if len(self.reverse_checks) else True
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

### BestFSFixedLossSelection
#### It inherits from the FixedRankingLossSimilarSelection. The BestFSFixedLossSelection strategy is a selection strategy that uses a variation of the best-first search algorithm to select the next item in the recommendation sequence. The strategy starts by initializing a priority queue and a set to keep track of the explored states. Each state corresponds to a binary mask representing the items that have been selected so far. The priority queue is initialized with a single state that corresponds to an empty mask.

#### At each step, the strategy materializes the current mask by computing the set of unmasked items and selecting the item that has the lowest loss. If the mask has not been explored before, the strategy adds it to the set of explored states, pushes it to the priority queue with a priority that is based on the loss of the selected item and a tie-breaking counter, and updates the best score for the current cardinality.

#### If the mask has been explored before, the strategy does not add it to the priority queue again but updates the best score for the current cardinality if the current score is better. The strategy then selects the state with the highest priority from the priority queue and materializes it to select the next item.

In [20]:
import heapq as hq


class BestFSFixedLossSelection(FixedRankingLossSimilarSelection):
    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model)

        self.best_score_per_cardinality = [-StaticVars.FLOAT_MAX] * self.max_length
        self.set = set()
        self.tiebraker = itertools.count()

#     is not currently used
#     def set_score(self, cardinality, target_score, kth_score):
#         is_updated = False
#         if self.best_score_per_cardinality[cardinality] > (target_score / kth_score):
#             self.best_score_per_cardinality[cardinality] = target_score / kth_score
#             is_updated = True

#         return is_updated

    def _materialize_list(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()

        if not self.is_materialized:
            if np.sum(m_mask) > 1:
                res = self.compute_losses(inv_mask=True)

                for idx in res:
                    m_mask[idx[1]] = False
                    self.reverse_checks.append((idx[0], next(self.tiebraker), m_mask.copy()))
                    self.set.add(int(''.join(map(str, m_mask.copy().astype(int))), 2))
                    m_mask[idx[1]] = True

                hq.heapify(self.reverse_checks)

            self.is_materialized = True

        _, _, m_mask = hq.heappop(self.reverse_checks) if len(self.reverse_checks) > 0 else (None, None, True)
        self.ma_arr = np.ma.masked_array(self.original_interactions, mask=m_mask)

        if np.sum(m_mask) > 1:
            m_mask = np.ma.getmask(self.ma_arr).copy()
            res = self.compute_losses(inv_mask=True)

            for idx in res:
                m_mask[idx[1]] = False
                self.add(m_mask.copy(), idx[0])
                m_mask[idx[1]] = True

    def add(self, d, pri):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if mask_to_int not in self.set:
            hq.heappush(self.reverse_checks, (pri, next(self.tiebraker), d))
            self.set.add(mask_to_int)

In [21]:
class BiDirectionalSelection(BaseStrategy):
    class_name = 'BiDirectional'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.tiebraker = itertools.count()
        self.q = [(1, StaticVars.INT_MAX, next(self.tiebraker), [False] * len(self.original_interactions), self.budget)]
        hq.heapify(self.q)

#         self.w_loss, self.w_custom = weights if len(weights) == 2 else (1, 0)
        self.alpha = alpha
        self.norm = normalization

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        is_solved, _, _, mask, budget = hq.heappop(self.q)
        return (is_solved, mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget
        if self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.search(forward=True, s=is_solved)
        self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_custom_score(self, c):
#         return self.w_custom * (c / self.max_length)
        return c / self.max_length

    def get_score(self, d):
        self.budget -= 1

        # predict next top-k items about to be selected
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))
        preds = self.model.predict(perm)

        if self.norm == 'kth_norm':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            t_score = preds[self.target_item] / preds[(rk_data == self.top_k).nonzero()][0]
        elif self.norm == 'rescale':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            max_val = rk_data[0]
            min_val = rk_data[-1]
            t_score = (max_val - preds[self.target_item]) / (max_val - min_val)
        else:  # default case
            tensor = F.softmax(torch.from_numpy(preds).float(), dim=0)
            preds = tensor.numpy()
            preds[perm] = -StaticVars.FLOAT_MAX

            t_score = preds[self.target_item]

        return self.alpha * t_score + (1 - self.alpha) * self.get_custom_score(np.sum(d))

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            t_score = self.get_score(d)
            hq.heappush(self.q, (int(not s), t_score, next(self.tiebraker), d.copy(), self.budget))

            self.visited_.add(mask_to_int)

In [22]:
class BruteForceSelection(BaseStrategy):
    class_name = 'BruteForce'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), self.budget))

    def _expand_queue(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask))[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask)
                m_mask[idx] = not m_mask[idx]

    def _next_item(self):
        mask, budget = self.q.dequeue()
        return (mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if reverse: self.q.clear()

        if self.q.size() > 0:
            item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._expand_queue()
        else:
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def add(self, d):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            self.budget -= 1
            self.q.enqueue((d.copy(), self.budget))
            self.visited_.add(mask_to_int)

In [23]:
class ComboSelection(BiDirectionalSelection):
    class_name = 'Combo'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model, weights, alpha, normalization)

        self.alpha = 1

        self.q_init = Queue()
        self.q_init.enqueue((StaticVars.INT_MAX, [False] * len(self.original_interactions), self.budget))
        self.init_queue()

        self.tiebraker = itertools.count()
        self.q = []
        hq.heapify(self.q)

        self.alpha = alpha

    def init_queue(self):
        _, m_mask, budget = self.q_init.dequeue()
        m_mask = np.asarray(m_mask)

        valid_items = np.where(np.logical_not(m_mask))[0]
        for idx in valid_items:
            m_mask[idx] = not m_mask[idx]

            mask_to_int = int(''.join(map(str, m_mask.astype(int))), 2)
            if (mask_to_int not in self.visited_) and (self.budget > 0):
                t_score = self.get_score(m_mask)
                self.q_init.enqueue((t_score, m_mask.copy(), self.budget))

                self.visited_.add(mask_to_int)

            m_mask[idx] = not m_mask[idx]

        pair_combs = []
        for c in itertools.combinations(range(len(self.original_interactions)), 2):
            m = [False] * len(self.original_interactions)
            m[c[0]], m[c[1]] = not m[c[0]], not m[c[1]]
            pair_combs.append((self.q_init.get(c[0])[0] + self.q_init.get(c[1])[0], m.copy()))

        pair_combs.sort(key=operator.itemgetter(0))
        for c in pair_combs:
            self.budget -= 1
            self.q_init.enqueue((0, c[1], self.budget))

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q_init.size() > 0:
            s, item_mask, budget = self.q_init.dequeue()
            item_mask = np.asarray(item_mask)
            solved_flag = False
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self.add(item_mask, False)
        elif self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

In [24]:
def get_backend_strategy(backend):
    if 'random' == backend:
        return RandomSelection
    elif 'most_sim' == backend:
        return MostSimilarSelection
    elif 'most_sim_jaccard' == backend:
        return MostSimilarSelectionByJaccard
    elif 'bfs' == backend:
        return LossSimilarSelection
    elif 'random_most_sim' == backend:
        return RandomMostSimilarSelection
    elif 'random_loss_sim' == backend:
        return RandomLossSimilarSelection
    elif 'fixed_loss_sim' == backend:
        return FixedRankingLossSimilarSelection
    elif 'dfs_loss_sim' == backend:
        return DFSwithLossSelection
    elif 'dfs_fixed_loss_sim' == backend:
        return DFSwithFixedRankingLossSelection
    elif 'bestFS_loss' == backend:
        return BestFSLossSelection
    elif 'bestFS_fixed_loss' == backend:
        return BestFSFixedLossSelection
    elif 'topdown_loss' == backend:
        return TopDownBestFSLossSelection
    elif 'bidirectional' == backend:
        return BiDirectionalSelection
    elif 'brute_force' == backend:
        return BruteForceSelection
    elif 'combo' == backend:
        return ComboSelection
    else: print('Unknown strategy')

# Execution of implemented strategies

## Strategies inputs

In [25]:
model_applied = 'lstm'

## **Random**

In [16]:
backend = 'random'
random_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='random', init_budget=1000)
]

%store random_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Random
<Sequence interactions dataset (13131 sequences x 20 sequence length)>


users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

user_id :  1
user_id :  2
user_id :  3
user_id :  4
user_id :  5
[-0.         -0.4886701  -1.2704934  ...  1.4422629   0.21140254
  0.10461938]
items_interacted :  [230 257 359 130 358 329 372 227 324 301 253 239 107 305 266 315  60 331
 123 357]
user_id :  6
user_id :  7
user_id :  8
[-0.        -1.7293239  0.7516235 ...  1.2451138  0.8454702  2.057247 ]
items_interacted :  [384 461 236 478  86 457 511  60 323 443 360 473 472 514 456 481 119 450
 453 444]
user_id :  9
[-0.         -0.01268113 -0.72315276 ...  1.8612478   1.2578342
  0.7910427 ]
items_interacted :  [ 24 105 126 523 268  22 337  40 506 318 317  90 539 227 273 114 549 323
 529  94]
user_id :  10
[-0.         -2.4513376   0.45527014 ...  1.0769624   1.3857057
  1.618092  ]
items_interacted :  [539 734 647 759 742  20 592 569 563  22  42 612 671  15 547 613  11 662
 435 660]


target position loop: 10it [00:07,  1.27it/s]              


KeyboardInterrupt: 

In [37]:
# Get the list of item IDs in the dataset
item_ids = list(dataset.item_ids)

# Set the number of items you want to select (e.g., 20)
num_items_to_select = 20

# Randomly select 20 item IDs from the list
selected_item_ids = random.sample(item_ids, num_items_to_select)



In [35]:
def find_best_items_with_jaccard(dataset, target_item, users, jaccard_sims_matrix, number_items=20):
    """
    Find the top best "number_items" for each user that, when added to their interactions list,
    have the highest Jaccard similarity to the target item based on the given similarity matrix.

    Parameters:
    - users: List of user indices.
    - jaccard_sims_matrix: Jaccard similarity matrix.
    - user_interactions: Dictionary where keys are user indices and values are their interaction lists.
    - number_items: The number of items to select for each user.

    Returns:
    - A dictionary where keys are user indices and values are lists of the top best items for each user.
    """
    
    best_items_for_users = {}

    for user_index in users:
        # Get the user's interactions and items not in interactions
        user_interactions_list = dataset.sequences[dataset.user_ids == user_index]
        
        items_not_in_interactions = np.setdiff1d(range(jaccard_sims_matrix.shape[1]), user_interactions_list)
        
        print(user_interactions_list)

        ranked_items = st.rankdata(jaccard_sims_matrix[user_interactions_list[0][target_item], items_not_in_interactions])

        # Get the top best items
        top_items_indices = np.argsort(ranked_items)[:number_items]

        best_items = [list(items_not_in_interactions)[idx] for idx in top_items_indices]

        best_items_for_users[user_index] = best_items

    return best_items_for_users

users_to_process = [1, 2, 3]  # Replace with your list of user indices
best_items_jaccard = find_best_items_with_jaccard(test, 2, users_to_process, jaccard_sims_matrix, 20)
print(best_items_jaccard)


[[ 0  0  0  0  0  0  0  0  0  0  0  0 45  3  7 21  6 34 11 36]]
[[  0   0 128 166 168 102  77  54 144 165 140  55 173  57  98  91 118  75
  155 127]]
[[  0   0   0   0   0   0   0   0   0 197 124 125 190 114 186 196  59 200
  191 177]]
{1: [1, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2466, 2478, 2480, 2481, 2482, 2483, 2484, 2485], 2: [3492, 3600, 3601, 3602, 3004, 3603, 3010, 3573, 3610, 3611, 3469, 1806, 3616, 3226, 3655, 3654, 3653, 3621, 3380, 3638], 3: [1, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2467, 2479, 2481, 2482, 2483, 2484, 2485, 2486]}


## **Most Similar**

In [359]:
cosine_on_embeddings_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000)
]
jaccard_on_embeddings_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
]

%store cosine_on_embeddings_cfs
%store jaccard_on_embeddings_cfs

## Utilize ylosses for similarities

In [104]:
backend = 'bfs'
bfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, early_term=True),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='loss_sim', init_budget=1000)
]

%store bfs_yloss_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BFS


target position loop: 20it [10:02, 30.12s/it]              

target position loop: 30it [16:37, 32.95s/it]

target position loop: 40it [21:32, 31.92s/it]

...

target position loop: 20it [07:38, 22.94s/it]              

target position loop: 30it [12:35, 24.95s/it]

target position loop: 40it [16:30, 24.51s/it]

target position loop: 40it [19:19, 28.98s/it]


Stored 'bfs_yloss_cfs' (list)


In [ ]:
print('Running BFS strategy with fixed ordering...')

bfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='fixed_loss_sim', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='fixed_loss_sim', init_budget=1000),
]

%store bfs_fixed_yloss_cfs

## Utilize similarities based on yloss with DFS backward search

In [232]:
backend='dfs_loss_sim'
dfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='dfs_loss_sim', init_budget=1000)
]

%store dfs_yloss_cfs

target position loop:   0%|          | 0/3 [00:00<?, ?it/s]

The backend used is: DFS


target position loop: 20it [00:28,  1.44s/it]              

target position loop: 30it [00:45,  1.49s/it]

target position loop: 30it [00:59,  1.97s/it]

Stored 'dfs_yloss_cfs' (list)


In [ ]:
backend='dfs_fixed_loss_sim'
dfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000)
]

%store dfs_fixed_yloss_cfs

In [ ]:
backend='bestFS_loss'
bestfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='bestFS_loss', init_budget=1000)
]

%store bestfs_yloss_cfs

In [ ]:
print('Running BestFS strategy with fixed ordering...')

bestfs_fixed_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='bestFS_fixed_loss', init_budget=1000),
    _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='bestFS_fixed_loss', init_budget=1000)
]

%store bestfs_fixed_yloss_cfs

In [ ]:
backend='topdown_loss'
topdown_bestfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [3, 5, 7], no_users=500, init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='topdown_loss', init_budget=1000)
]

%store topdown_bestfs_yloss_cfs

In [75]:
backend='bidirectional'
bidirectional_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.999, normalization='default'),
]

%store bidirectional_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BiDirectional


users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 20it [1:44:19, 312.97s/it]           

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [3:58:44, 389.55s/it]

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [5:18:24, 477.60s/it]


Stored 'bidirectional_cfs' (list)


In [72]:
backend='brute_force'
brute_force_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=100000),
]

%store brute_force_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BruteForce


target position loop: 20it [2:59:30, 538.52s/it]           

target position loop: 30it [4:11:08, 505.90s/it]

target position loop: 40it [4:28:50, 385.98s/it]

target position loop: 40it [4:33:42, 410.56s/it]


Stored 'brute_force_cfs' (list)


In [77]:
backend='combo'
combo_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [1, 3, 5, 7], no_users=6000, init_budget=1000, alpha=0.999, normalization='default'),
]

%store combo_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Combo


users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 20it [2:34:47, 464.38s/it]           

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

target position loop: 40it [5:18:15, 519.91s/it]

users loop:   0%|          | 0/6000 [00:00<?, ?it/s]

target position loop: 40it [7:05:47, 638.69s/it]


Stored 'combo_cfs' (list)
